----
## VGG16 Architecture - First Pass Training

In [1]:
import numpy as np
import pandas as pd
from subprocess import check_output

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import pickle

from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers import (AveragePooling2D, Convolution2D, Dense, Dropout,
    Flatten, Input, MaxPooling2D, ZeroPadding2D)
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model, Sequential
from keras import optimizers
from keras.utils import to_categorical
from keras.models import load_model

import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Load dataset for input to model
HH_HV_Com_75 = pickle.load(open('HH_HV_Com_75.p','rb'))
HH_HV_Com_75.shape
df_train = pd.read_json('train.json')
y = np.array(df_train.is_iceberg)
y = to_categorical(y)

### VGG16 Slightly Modified Architecture

Small changes made to VGG16 architecture:<br>
    Input datasize of 75x75 pixels <br>
    Data augmentation with zoom and rotation to expand training dataset size <br>
    Trained for a total of 70 Epochs <br>
    Dropout of .1 added to last two FC layers <br>


In [7]:
# Set batch size and data augmentation parameters
batch_size=96
gen = ImageDataGenerator(horizontal_flip = False,
                         vertical_flip = False,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 40,
                        fill_mode = 'wrap')

def Model():
    vgg_model = Sequential()
    vgg_model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=(75,75,3), border_mode='same', name='conv1_1'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv1_1_zp'))
    vgg_model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
    vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool1'))

    vgg_model.add(ZeroPadding2D((1, 1), name='pool1_zp'))
    vgg_model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv2_1_zp'))
    vgg_model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
    vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool2'))

    vgg_model.add(ZeroPadding2D((1, 1), name='pool2_zp'))
    vgg_model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv3_1_zp'))
    vgg_model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv3_2_zp'))
    vgg_model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
    vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool3'))

    vgg_model.add(ZeroPadding2D((1, 1), name='conv4_1_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv4_2_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
    vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool4'))

    vgg_model.add(ZeroPadding2D((1, 1), name='pool4_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv5_1_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv5_2_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
    vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool5'))
    
    vgg_model.add(ZeroPadding2D((1, 1), name='pool5_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv6_1'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv6_1_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv6_2'))
    vgg_model.add(ZeroPadding2D((1, 1), name='conv6_2_zp'))
    vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv6_3'))
    vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool6'))

    vgg_model.add(Flatten())
    vgg_model.add(Dense(4096, activation='relu', name='fc6'))
    vgg_model.add(Dropout(0.1, name='fc6do'))
    vgg_model.add(Dense(4096, activation='relu', name='fc7'))
    vgg_model.add(Dropout(0.1, name='fc7do'))
    vgg_model.add(Dense(2, activation='softmax', name='predictions'))
    vgg_model.summary();
    
    adam = optimizers.adam(lr=.0001)
    vgg_model.compile(optimizer=adam,
    loss='binary_crossentropy', metrics=['accuracy'])
    
    return vgg_model

def train_model(X_train, X_test, y_train, y_test):
    
    gen_flow = gen.flow(X_train,y_train,batch_size=batch_size,seed=42)
    vgg_Model= Model()
    vgg_Model.fit_generator(
            gen_flow,
            steps_per_epoch=12,
            epochs=50,
            shuffle=True,
            verbose=1,
            validation_data=(X_test, y_test))
    global vgg_Model
    
def continue_training(X_train, X_test, y_train, y_test):
    
    model = load_model('models/vgg16.h5')
    gen_flow = gen.flow(X_train,y_train,batch_size=batch_size,seed=42)
    vgg_Model = model
    vgg_Model.fit_generator(
            gen_flow,
            steps_per_epoch=12,
            epochs=20,
            shuffle=True,
            verbose=1,
            validation_data=(X_test, y_test))
    global vgg_Model

In [4]:
# Split test and train
X_train, X_test, y_train, y_test = train_test_split(
    HH_HV_Com_75, y, test_size=0.20, random_state=42)

In [ ]:
train_model(X_train, X_test, y_train, y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 75, 75, 64)        1792      
_________________________________________________________________
conv1_1_zp (ZeroPadding2D)   (None, 77, 77, 64)        0         
_________________________________________________________________
conv1_2 (Conv2D)             (None, 75, 75, 64)        36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 37, 37, 64)        0         
_________________________________________________________________
pool1_zp (ZeroPadding2D)     (None, 39, 39, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 37, 37, 128)       73856     
_________________________________________________________________
conv2_1_zp (ZeroPadding2D)   (None, 39, 39, 128)       0         
__________

In [9]:
# Save model
vgg_Model.save('models/vgg16.h5')

In [8]:
# Continue training from known model location
continue_training(X_train, X_test, y_train, y_test)

Epoch 1/20
14/12 [===================================] - 83s 6s/step - loss: 0.2394 - acc: 0.8891 - val_loss: 0.2517 - val_acc: 0.8847
Epoch 2/20
14/12 [===================================] - 82s 6s/step - loss: 0.2178 - acc: 0.9055 - val_loss: 0.2947 - val_acc: 0.8505
Epoch 3/20
14/12 [===================================] - 82s 6s/step - loss: 0.2509 - acc: 0.8750 - val_loss: 0.2549 - val_acc: 0.8847
Epoch 4/20
14/12 [===================================] - 84s 6s/step - loss: 0.2224 - acc: 0.9030 - val_loss: 0.2627 - val_acc: 0.8910
Epoch 5/20
14/12 [===================================] - 84s 6s/step - loss: 0.2484 - acc: 0.8879 - val_loss: 0.2976 - val_acc: 0.8411
Epoch 6/20
14/12 [===================================] - 85s 6s/step - loss: 0.2675 - acc: 0.8798 - val_loss: 0.3401 - val_acc: 0.8910
Epoch 7/20
14/12 [===================================] - 83s 6s/step - loss: 0.2589 - acc: 0.8774 - val_loss: 0.3263 - val_acc: 0.8847
Epoch 8/20
14/12 [===================================] 